In [44]:
from __future__ import annotations
import requests
from datetime import date
import pandas as pd
import numpy as np
import io
import zipfile
from pathlib import Path
from typing import Optional, Union, List, Dict, Any

In [45]:
BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"

def _fmt_mmddyyyy(d: date) -> str:
    return d.strftime("%m-%d-%Y")

def cotacao_dolar_periodo_df(data_ini: date, data_fim: date) -> pd.DataFrame:
    url = f"{BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    params = {
        "@dataInicial": f"'{_fmt_mmddyyyy(data_ini)}'",
        "@dataFinalCotacao": f"'{_fmt_mmddyyyy(data_fim)}'",
        "$format": "json",
        "$select": "cotacaoCompra,cotacaoVenda,dataHoraCotacao",
        "$top": 10000,
    }

    r = requests.get(url, params=params, timeout=30)
    if not r.ok:
        raise requests.HTTPError(f"{r.status_code} - {r.text}", response=r)

    payload = r.json()
    df = pd.DataFrame(payload.get("value", []))

    if df.empty:
        return df

    df["dataHoraCotacao"] = pd.to_datetime(df["dataHoraCotacao"])
    df = df.sort_values("dataHoraCotacao").reset_index(drop=True)
    return df

def dolar_diario(df_ptax: pd.DataFrame) -> pd.DataFrame:
    """Converte boletins intradiários em série diária (média por dia)."""
    if df_ptax.empty:
        return df_ptax

    df = df_ptax.copy()
    df["data"] = df["dataHoraCotacao"].dt.date
    out = (
        df.groupby("data", as_index=False)[["cotacaoCompra", "cotacaoVenda"]]
          .mean()
          .rename(columns={"cotacaoCompra": "compra", "cotacaoVenda": "venda"})
    )
    out["data"] = pd.to_datetime(out["data"])
    return out.sort_values("data").reset_index(drop=True)


In [46]:
def processar_scrdata(
    df: pd.DataFrame,
    remover_zeros: bool = True,
    criar_indicadores: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Processa automaticamente dados do SCR.data para análise e visualização.

    Etapas:
    - padroniza nomes das colunas
    - converte datas
    - converte valores monetários
    - cria colunas temporais
    - cria indicadores financeiros
    - valida consistência
    """

    df = df.copy()

    if verbose:
        print("🔄 Iniciando processamento do SCR.data...")

    # ------------------------------------------------
    # 1. Padronizar nomes das colunas
    # ------------------------------------------------
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("ç", "c")
        .str.replace("ã", "a")
        .str.replace("á", "a")
        .str.replace("é", "e")
        .str.replace("í", "i")
        .str.replace("ó", "o")
        .str.replace("ú", "u")
    )

    # ------------------------------------------------
    # 2. Converter data
    # ------------------------------------------------
    if "data_base" in df.columns:
        df["data_base"] = pd.to_datetime(df["data_base"], errors="coerce")

        df["ano"] = df["data_base"].dt.year
        df["mes"] = df["data_base"].dt.month

    # ------------------------------------------------
    # 3. Converter valores monetários
    # Detecta automaticamente colunas numéricas
    # ------------------------------------------------
    colunas_texto = [
        "uf", "segmento", "cliente", "cnae_ocupacao",
        "porte", "modalidade", "submodalidade",
        "origem", "indexador"
    ]

    colunas_para_converter = [
        c for c in df.columns
        if c not in colunas_texto
        and c not in ["data_base", "ano", "mes", "__arquivo_origem"]
    ]

    for c in colunas_para_converter:
        if df[c].dtype == "object":
            try:
                df[c] = (
                    df[c]
                    .str.replace(".", "", regex=False)
                    .str.replace(",", ".", regex=False)
                )
                df[c] = pd.to_numeric(df[c], errors="coerce")
            except Exception:
                pass

    # ------------------------------------------------
    # 4. Criar indicadores financeiros
    # ------------------------------------------------
    if criar_indicadores:

        if {"carteira_inadimplencia", "carteira_ativa"}.issubset(df.columns):
            df["taxa_inadimplencia"] = (
                df["carteira_inadimplencia"] /
                df["carteira_ativa"]
            ).replace([np.inf, -np.inf], np.nan)

        if {"carteira_vencida", "carteira_ativa"}.issubset(df.columns):
            df["perc_carteira_vencida"] = (
                df["carteira_vencida"] /
                df["carteira_ativa"]
            ).replace([np.inf, -np.inf], np.nan)

        if {"ativo_problematico", "carteira_ativa"}.issubset(df.columns):
            df["taxa_ativo_problematico"] = (
                df["ativo_problematico"] /
                df["carteira_ativa"]
            ).replace([np.inf, -np.inf], np.nan)

    # ------------------------------------------------
    # 5. Remover registros sem valor analítico
    # ------------------------------------------------
    if remover_zeros and "carteira_ativa" in df.columns:
        antes = len(df)
        df = df[df["carteira_ativa"] > 0]
        depois = len(df)

        if verbose:
            print(f"🧹 Removidas {antes - depois:,} linhas com carteira zerada.")

    # ------------------------------------------------
    # 6. Checagem de consistência
    # ------------------------------------------------
    if {"carteira_ativa", "carteira_vencida"}.issubset(df.columns):
        inconsistentes = df[df["carteira_vencida"] > df["carteira_ativa"]]

        if verbose and len(inconsistentes) > 0:
            print(f"⚠️ {len(inconsistentes)} registros inconsistentes encontrados.")

    # ------------------------------------------------
    # 7. Ordenar dataset
    # ------------------------------------------------
    if "data_base" in df.columns:
        df = df.sort_values("data_base")

    # ------------------------------------------------
    # 8. Reset index
    # ------------------------------------------------
    df = df.reset_index(drop=True)

    if verbose:
        print("✅ SCR.data processado com sucesso!")
        print(f"Linhas: {len(df):,}")
        print(f"Colunas: {len(df.columns)}")

    return df

In [47]:
def pipeline_scrdata(
    ano: int,
    base_dir: Union[str, Path] = "data/scrdata",
    forcar_download: bool = False,
    encoding: str = "utf-8",
    sep: Optional[str] = None,
    salvar_parquet: bool = True,
    salvar_csv: bool = False,
    adicionar_coluna_origem: bool = True,
    remover_zeros: bool = True,
    criar_indicadores: bool = True,
    verbose: bool = True,
) -> Dict[str, Any]:
    """
    Pipeline completo do SCR.data:
    - baixa ZIP por ano (com cache)
    - extrai (com cache)
    - concatena todos os CSVs mensais
    - processa (tipos, datas, indicadores)
    - salva em parquet/csv
    - retorna artefatos e DataFrame

    Retorna um dicionário com:
      - df_raw, df_processed
      - paths (zip, extracted_dir, parquet/csv)
      - metadata
    """
    if not (1900 <= int(ano) <= 2100):
        raise ValueError(f"Ano inválido: {ano}")

    base_dir = Path(base_dir)
    ano_dir = base_dir / str(ano)
    raw_dir = ano_dir / "raw"
    extracted_dir = ano_dir / "extracted"
    processed_dir = ano_dir / "processed"

    raw_dir.mkdir(parents=True, exist_ok=True)
    extracted_dir.mkdir(parents=True, exist_ok=True)
    processed_dir.mkdir(parents=True, exist_ok=True)

    url = f"https://www.bcb.gov.br/pda/desig/scrdata_{ano}.zip"
    zip_path = raw_dir / f"scrdata_{ano}.zip"

    # ---------------------------
    # 1) Download (cache por ano)
    # ---------------------------
    if zip_path.exists() and not forcar_download:
        if verbose:
            print(f"ℹ️ ZIP {ano} já existe: {zip_path.resolve()}")
            print("➡️ Download não será realizado.")
    else:
        if verbose:
            print(f"⬇️ Baixando: {url}")
        r = requests.get(url, timeout=180)
        r.raise_for_status()
        zip_path.write_bytes(r.content)
        if verbose:
            print(f"✅ ZIP salvo em: {zip_path.resolve()}")

    # ---------------------------
    # 2) Extração (cache)
    # ---------------------------
    ja_extraido = any(extracted_dir.rglob("*.csv"))
    if ja_extraido and not forcar_download:
        if verbose:
            print(f"ℹ️ CSVs já extraídos em: {extracted_dir.resolve()}")
            print("➡️ Extração não será realizada.")
    else:
        if forcar_download:
            # limpa extração para evitar mistura
            for p in sorted(extracted_dir.rglob("*"), reverse=True):
                if p.is_file():
                    p.unlink()
                elif p.is_dir():
                    try:
                        p.rmdir()
                    except OSError:
                        pass

        if verbose:
            print(f"🗜️ Extraindo para: {extracted_dir.resolve()}")
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extracted_dir)
        if verbose:
            print("✅ Extração concluída.")

    # ---------------------------
    # 3) Localizar CSVs
    # ---------------------------
    csv_paths = sorted(extracted_dir.rglob("*.csv"))
    if not csv_paths:
        raise FileNotFoundError(f"Nenhum CSV encontrado em {extracted_dir.resolve()}")

    if verbose:
        print(f"📄 CSVs encontrados: {len(csv_paths)}")

    # ---------------------------
    # 4) Ler e concatenar CSVs
    # ---------------------------
    def inferir_sep(csv_path: Path, enc: str) -> str:
        sample = csv_path.read_bytes()[:50_000]
        text = sample.decode(enc, errors="ignore")
        return ";" if text.count(";") > text.count(",") else ","

    dfs: List[pd.DataFrame] = []
    for p in csv_paths:
        local_sep = sep or inferir_sep(p, encoding)

        try:
            df_part = pd.read_csv(p, encoding=encoding, sep=local_sep)
        except UnicodeDecodeError:
            df_part = pd.read_csv(p, encoding="latin1", sep=local_sep)

        if adicionar_coluna_origem:
            df_part["__arquivo_origem"] = p.name

        dfs.append(df_part)

    df_raw = pd.concat(dfs, ignore_index=True, sort=False)

    if verbose:
        print(f"✅ RAW consolidado: {len(df_raw):,} linhas | {len(df_raw.columns)} colunas")

    # ---------------------------
    # 5) Processar (usa sua função)
    # ---------------------------
    # Requer que processar_scrdata(df) exista no notebook.
    df_processed = processar_scrdata(
        df_raw,
        remover_zeros=remover_zeros,
        criar_indicadores=criar_indicadores,
        verbose=verbose
    )

    # ---------------------------
    # 6) Salvar outputs
    # ---------------------------
    parquet_path = processed_dir / f"scrdata_{ano}.parquet"
    csv_path = processed_dir / f"scrdata_{ano}.csv"

    if salvar_parquet:
        df_processed.to_parquet(parquet_path, index=False)
        if verbose:
            print(f"💾 Parquet salvo em: {parquet_path.resolve()}")

    if salvar_csv:
        df_processed.to_csv(csv_path, index=False, encoding="utf-8-sig")
        if verbose:
            print(f"💾 CSV salvo em: {csv_path.resolve()}")

    # Metadados
    df_processed.attrs["fonte_url"] = url
    df_processed.attrs["zip_path"] = str(zip_path.resolve())
    df_processed.attrs["extracted_dir"] = str(extracted_dir.resolve())
    df_processed.attrs["parquet_path"] = str(parquet_path.resolve()) if salvar_parquet else None
    df_processed.attrs["csv_path"] = str(csv_path.resolve()) if salvar_csv else None
    df_processed.attrs["csv_count"] = len(csv_paths)

    return {
        "df_raw": df_raw,
        "df_processed": df_processed,
        "paths": {
            "zip": zip_path.resolve(),
            "extracted_dir": extracted_dir.resolve(),
            "parquet": parquet_path.resolve() if salvar_parquet else None,
            "csv": csv_path.resolve() if salvar_csv else None,
        },
        "metadata": {
            "ano": ano,
            "url": url,
            "csv_count": len(csv_paths),
            "rows_raw": len(df_raw),
            "rows_processed": len(df_processed),
            "cols_processed": len(df_processed.columns),
        }
    }

In [48]:
df_periodo = cotacao_dolar_periodo_df(date(2026, 1, 1), date(2026, 2, 5))
print(df_periodo)

    cotacaoCompra  cotacaoVenda         dataHoraCotacao
0          5.4366        5.4372 2026-01-02 13:03:33.254
1          5.4345        5.4351 2026-01-05 13:10:14.896
2          5.3791        5.3797 2026-01-06 13:12:07.358
3          5.3874        5.3880 2026-01-07 13:06:27.001
4          5.3854        5.3860 2026-01-08 13:04:28.616
5          5.3701        5.3707 2026-01-09 13:09:29.054
6          5.3754        5.3760 2026-01-12 13:08:29.056
7          5.3758        5.3764 2026-01-13 13:08:54.165
8          5.3789        5.3795 2026-01-14 13:04:32.411
9          5.3840        5.3846 2026-01-15 13:05:25.660
10         5.3792        5.3798 2026-01-16 13:04:26.486
11         5.3647        5.3653 2026-01-19 13:07:27.662
12         5.3784        5.3790 2026-01-20 13:02:30.787
13         5.3362        5.3368 2026-01-21 13:04:32.866
14         5.3112        5.3118 2026-01-22 13:07:26.401
15         5.2872        5.2879 2026-01-23 13:09:25.549
16         5.2754        5.2760 2026-01-26 13:09

In [49]:
resultado = pipeline_scrdata(2025, salvar_csv=False, salvar_parquet=True)

df_scr_2025 = resultado["df_processed"]
resultado["paths"]

ℹ️ ZIP 2025 já existe: C:\Users\DEINF.LEOBARROS\Desktop\Cursos\Pós\pos_machine_learning\Projeto_Final\data\scrdata\2025\raw\scrdata_2025.zip
➡️ Download não será realizado.
ℹ️ CSVs já extraídos em: C:\Users\DEINF.LEOBARROS\Desktop\Cursos\Pós\pos_machine_learning\Projeto_Final\data\scrdata\2025\extracted
➡️ Extração não será realizada.
📄 CSVs encontrados: 10
✅ RAW consolidado: 3,066,086 linhas | 25 colunas
🔄 Iniciando processamento do SCR.data...
🧹 Removidas 0 linhas com carteira zerada.
✅ SCR.data processado com sucesso!
Linhas: 3,066,086
Colunas: 30
💾 Parquet salvo em: C:\Users\DEINF.LEOBARROS\Desktop\Cursos\Pós\pos_machine_learning\Projeto_Final\data\scrdata\2025\processed\scrdata_2025.parquet


{'zip': WindowsPath('C:/Users/DEINF.LEOBARROS/Desktop/Cursos/Pós/pos_machine_learning/Projeto_Final/data/scrdata/2025/raw/scrdata_2025.zip'),
 'extracted_dir': WindowsPath('C:/Users/DEINF.LEOBARROS/Desktop/Cursos/Pós/pos_machine_learning/Projeto_Final/data/scrdata/2025/extracted'),
 'parquet': WindowsPath('C:/Users/DEINF.LEOBARROS/Desktop/Cursos/Pós/pos_machine_learning/Projeto_Final/data/scrdata/2025/processed/scrdata_2025.parquet'),
 'csv': None}

In [50]:
codigo = """
import streamlit as st
import pandas as pd
import requests
from datetime import date
import plotly.graph_objects as go
from scr_pipeline import pipeline_scrdata

# ==============================
# PTAX - DÓLAR
# ==============================
BASE = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata"

def _fmt_mmddyyyy(d: date) -> str:
    return d.strftime("%m-%d-%Y")

@st.cache_data(ttl=3600)
def cotacao_dolar_periodo_df(data_ini: date, data_fim: date) -> pd.DataFrame:
    url = f"{BASE}/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    params = {
        "@dataInicial": f"'{_fmt_mmddyyyy(data_ini)}'",
        "@dataFinalCotacao": f"'{_fmt_mmddyyyy(data_fim)}'",
        "$format": "json",
        "$select": "cotacaoCompra,cotacaoVenda,dataHoraCotacao",
        "$top": 10000,
    }

    r = requests.get(url, params=params, timeout=30)
    if not r.ok:
        raise requests.HTTPError(f"{r.status_code} - {r.text}", response=r)

    payload = r.json()
    df = pd.DataFrame(payload.get("value", []))

    if df.empty:
        return df

    df["dataHoraCotacao"] = pd.to_datetime(df["dataHoraCotacao"])
    df = df.sort_values("dataHoraCotacao").reset_index(drop=True)
    return df


def dolar_diario(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    df["data"] = df["dataHoraCotacao"].dt.date
    out = (
        df.groupby("data", as_index=False)[["cotacaoCompra", "cotacaoVenda"]]
          .mean()
          .rename(columns={
              "cotacaoCompra": "compra",
              "cotacaoVenda": "venda"
          })
    )
    out["data"] = pd.to_datetime(out["data"])
    return out.sort_values("data").reset_index(drop=True)


# ==============================
# SCR
# ==============================
@st.cache_data(show_spinner=True)
def carregar_scr(ano: int) -> pd.DataFrame:
    resultado = pipeline_scrdata(
        ano,
        salvar_parquet=True,
        salvar_csv=False,
        verbose=False
    )
    return resultado["df_processed"]


# ==============================
# UI
# ==============================
st.set_page_config(
    page_title="Painel BCB - Dólar e SCR",
    layout="wide"
)

st.title("📊 Painel BCB — Dólar (PTAX) e SCR.data")

with st.sidebar:
    st.header("Fonte de dados")
    fonte = st.selectbox(
        "Escolha o dado",
        ["Dólar comercial (PTAX)", "SCR.data"]
    )


# ==============================
# DÓLAR
# ==============================
if fonte == "Dólar comercial (PTAX)":

    st.subheader("💵 Dólar comercial — PTAX")

    hoje = date.today()

    with st.sidebar:
        st.header("Parâmetros")
        data_ini = st.date_input("Data inicial", value=date(hoje.year, 1, 1))
        data_fim = st.date_input("Data final", value=hoje)
        consultar = st.button("Consultar")

    if consultar:
        if data_ini > data_fim:
            st.error("A data inicial não pode ser maior que a data final.")
            st.stop()

        with st.spinner("Consultando API do Banco Central..."):
            df_raw = cotacao_dolar_periodo_df(data_ini, data_fim)
            df = dolar_diario(df_raw)

        if df.empty:
            st.warning("Nenhuma cotação encontrada para o período.")
            st.stop()

        ultima = df.iloc[-1]
        c1, c2 = st.columns(2)
        c1.metric("Última compra", f"R$ {ultima['compra']:.4f}")
        c2.metric("Última venda", f"R$ {ultima['venda']:.4f}")

        st.subheader("📈 Evolução diária do dólar")
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df["data"], y=df["compra"], name="Compra"))
        fig.add_trace(go.Scatter(x=df["data"], y=df["venda"], name="Venda"))
        fig.update_layout(
            height=520,
            xaxis_title="Data",
            yaxis_title="R$",
            margin=dict(l=10, r=10, t=40, b=10)
        )
        st.plotly_chart(fig, use_container_width=True)

        st.dataframe(df, use_container_width=True)

        st.download_button(
            "⬇️ Baixar CSV",
            data=df.to_csv(index=False).encode("utf-8"),
            file_name="cotacao_dolar_ptax.csv",
            mime="text/csv"
        )

    else:
        st.info("Selecione o período e clique em **Consultar**.")


# ==============================
# SCR
# ==============================
else:
    st.subheader("📊 SCR.data — Sistema de Informações de Crédito")

    with st.sidebar:
        ano = st.number_input("Ano", min_value=2010, max_value=2100, value=2025, step=1)

    df = carregar_scr(int(ano))

    if df.empty:
        st.warning("Nenhum dado disponível para o ano selecionado.")
        st.stop()

    df["data_base"] = pd.to_datetime(df["data_base"])

    with st.sidebar:
        st.header("Filtros")
        uf = st.selectbox("UF", ["(Todas)"] + sorted(df["uf"].dropna().unique()))
        cliente = st.selectbox("Cliente", ["(Todos)"] + sorted(df["cliente"].dropna().unique()))
        modalidade = st.selectbox("Modalidade", ["(Todas)"] + sorted(df["modalidade"].dropna().unique()))

    df_f = df.copy()
    if uf != "(Todas)": df_f = df_f[df_f["uf"] == uf]
    if cliente != "(Todos)": df_f = df_f[df_f["cliente"] == cliente]
    if modalidade != "(Todas)": df_f = df_f[df_f["modalidade"] == modalidade]

    st.caption(f"Registros após filtros: {len(df_f):,}")

    c1, c2, c3 = st.columns(3)
    c1.metric("Carteira ativa", f"{df_f['carteira_ativa'].sum():,.2f}")
    c2.metric("Inadimplência", f"{df_f['carteira_inadimplencia'].sum():,.2f}")

    taxa = (
        (df_f["taxa_inadimplencia"] * df_f["carteira_ativa"]).sum()
        / df_f["carteira_ativa"].sum()
        if df_f["carteira_ativa"].sum() > 0 else 0
    )
    c3.metric("Taxa inadimplência", f"{taxa:.2%}")

    st.subheader("📈 Evolução da carteira ativa")
    serie_ativa = df_f.groupby("data_base")["carteira_ativa"].sum()
    st.line_chart(serie_ativa, use_container_width=True)

    st.subheader("📈 Evolução da inadimplência")
    serie_inad = df_f.groupby("data_base")["carteira_inadimplencia"].sum()
    st.line_chart(serie_inad, use_container_width=True)

    st.subheader("📊 Top modalidades por carteira ativa")
    top_mod = df_f.groupby("modalidade")["carteira_ativa"].sum().sort_values(ascending=False).head(15)
    st.bar_chart(top_mod, use_container_width=True)

    st.subheader("📊 Top UFs por carteira ativa")
    top_uf = df_f.groupby("uf")["carteira_ativa"].sum().sort_values(ascending=False).head(15)
    st.bar_chart(top_uf, use_container_width=True)

    st.subheader("📥 Exportar dados filtrados")
    st.download_button(
        "Baixar CSV",
        data=df_f.to_csv(index=False).encode("utf-8"),
        file_name=f"scr_filtrado_{ano}.csv",
        mime="text/csv"
    )
"""

In [51]:
with open("app.py", "w", encoding="utf-8") as f:
    f.write(codigo)

print("✅ app.py criado com sucesso!")

✅ app.py criado com sucesso!
